In [3]:
import numpy as np
print(np.float32(4163.0))
table={}
table[0]="a"
print(table)


4163.0
{0: 'a'}


In [2]:
import json
with open("../../IM-TQA/data/train_tables copy.json", "r") as f:
    data1 = json.load(f)
total1=0
total2=0
total3=0
total4=0
for each in data1:
    total1+=len(each["column_attribute"])
    total2+=len(each["row_attribute"])
    total3+=len(each["column_index"])
    total4+=len(each["row_index"])
total5=29205-total1-total2-total3-total4
print(total1,total2,total3,total4,total5)
# with open("../../IM-TQA/data/dev_tables.json", "r") as f:
#     data2 = json.load(f)
# total1=0
# total2=0
# total3=0
# total4=0
# for each in data2:
#     total1+=len(each["column_attribute"])
#     total2+=len(each["row_attribute"])
#     total3+=len(each["column_index"])
#     total4+=len(each["row_index"])
# total5=2952-total1-total2-total3-total4
# print(total1,total2,total3,total4,total5)
with open("../../table_data/test_data.json", "r") as f:
    data3 = json.load(f)
data = data1 + data3
print(len(data1))
print(len(data3))
print(len(data))
with open("../../table_data/train_dev_test_new.json", "w") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

3836 4905 989 2351 17124
936
364
1300


In [10]:
import json
jdl=[]
with open("../../table_data/01-10.json","r") as f:
    jdl=json.load(f)
print("01-10:", len(jdl))
with open("../../table_data/神通.json", "r") as f:
    jdl = json.load(f)
print("神通:",len(jdl))
with open("../../table_data/GBase.json", "r") as f:
    jdl = json.load(f)
print("GBase:",len(jdl))
with open("../../table_data/King1.json", "r") as f:
    jdl = json.load(f)
print("King1:",len(jdl))
with open("../../table_data/King2.json", "r") as f:
    jdl = json.load(f)
print("King2:",len(jdl))
with open("../../table_data/Ocean.json", "r") as f:
    jdl = json.load(f)
print("Ocean:",len(jdl))

01-10: 219
神通: 105
GBase: 707
King1: 9
King2: 30
Ocean: 265


In [2]:
import json
with open("../../IM-TQA/llm_outputs/GPT_3.5_zero_shot.json","r") as f:
    jdl=json.load(f)
print("GPT_3.5_zero_shot:", len(jdl))

GPT_3.5_zero_shot: 622


In [4]:
with open("../../IM-TQA/data/dev_questions.json","r") as f:
    jdl=json.load(f)
print("dev_questions:", len(jdl))
with open("../../IM-TQA/data/train_questions.json","r") as f:
    jdl=json.load(f)
print("train_questions:", len(jdl))
with open("../../IM-TQA/data/test_questions.json","r") as f:
    jdl=json.load(f)
print("test_questions:", len(jdl))

dev_questions: 464
train_questions: 3909
test_questions: 627


In [1]:
print("| 类别 | 年末余额 | 年末余额 | 年末余额 | 年末余额 | 年末余额 |\n| 类别 | 账面余额 | 账面余额 | 坏账准备 | 坏账准备 | 账面价值 |\n| 类别 | 金额 | 比例（%） | 金额 | 计提比例（%） | 账面价值 |\n| 按单项计提应收账款坏账准备 | 693,480.70 | 34.23 |  |  | 693,480.70 |\n| 按组合计提应收账款坏账准备 | 1,332,684.97  |   65.77  |    132,849.18  |    9.97  |  1,199,835.79 |\n| 合计 | 2,026,165.67  |   100.00  |    132,849.18  | — | 1,893,316.49 |")


| 类别 | 年末余额 | 年末余额 | 年末余额 | 年末余额 | 年末余额 |
| 类别 | 账面余额 | 账面余额 | 坏账准备 | 坏账准备 | 账面价值 |
| 类别 | 金额 | 比例（%） | 金额 | 计提比例（%） | 账面价值 |
| 按单项计提应收账款坏账准备 | 693,480.70 | 34.23 |  |  | 693,480.70 |
| 按组合计提应收账款坏账准备 | 1,332,684.97  |   65.77  |    132,849.18  |    9.97  |  1,199,835.79 |
| 合计 | 2,026,165.67  |   100.00  |    132,849.18  | — | 1,893,316.49 |


In [1]:
import os
import sys
import time
import argparse
import json
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

def load_table_data(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    json_dict = data[0]
    table = pd.DataFrame(
        index=range(len(json_dict["cell_ID_matrix"])),
        columns=range(len(json_dict["cell_ID_matrix"][0])),
    )
    i = 0
    for row in json_dict["cell_ID_matrix"]:
        table.iloc[i] = [each for each in row]
        i = i + 1
    nodes = []
    for i in range(len(table)):
        for j in range(len(table.columns)):
            node = table.iloc[i, j]
            if node not in nodes:
                nodes.append(node)
    relations = pd.DataFrame(np.zeros((len(nodes), len(nodes))))
    for i in range(len(table)):
        for j in range(len(table.columns)):
            node = table.iloc[i, j]
            if i >= 0 and j - 1 >= 0 and i < len(table) and j - 1 < len(table.columns):
                if node != table.iloc[i, j - 1]:
                    relations.iloc[node, table.iloc[i, j - 1]] = 1
            if i >= 0 and j + 1 >= 0 and i < len(table) and j + 1 < len(table.columns):
                if node != table.iloc[i, j + 1]:
                    relations.iloc[node, table.iloc[i, j + 1]] = 2
            if i - 1 >= 0 and j >= 0 and i - 1 < len(table) and j < len(table.columns):
                if node != table.iloc[i - 1, j]:
                    relations.iloc[node, table.iloc[i - 1, j]] = 3
            if i + 1 >= 0 and j >= 0 and i + 1 < len(table) and j < len(table.columns):
                if node != table.iloc[i + 1, j]:
                    relations.iloc[node, table.iloc[i + 1, j]] = 4
    print(relations)
    csr_relations = csr_matrix(relations.values)
    print(csr_relations)
load_table_data("../../IM-TQA/data/train_tables.json")

     0    1    2    3    4    5    6    7    8    9
0  0.0  2.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0
1  1.0  0.0  2.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0
2  0.0  1.0  0.0  2.0  0.0  0.0  0.0  4.0  0.0  0.0
3  0.0  0.0  1.0  0.0  2.0  0.0  0.0  0.0  4.0  0.0
4  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  4.0
5  3.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0
6  0.0  3.0  0.0  0.0  0.0  1.0  0.0  2.0  0.0  0.0
7  0.0  0.0  3.0  0.0  0.0  0.0  1.0  0.0  2.0  0.0
8  0.0  0.0  0.0  3.0  0.0  0.0  0.0  1.0  0.0  2.0
9  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  1.0  0.0
  (0, 1)	2.0
  (0, 5)	4.0
  (1, 0)	1.0
  (1, 2)	2.0
  (1, 6)	4.0
  (2, 1)	1.0
  (2, 3)	2.0
  (2, 7)	4.0
  (3, 2)	1.0
  (3, 4)	2.0
  (3, 8)	4.0
  (4, 3)	1.0
  (4, 9)	4.0
  (5, 0)	3.0
  (5, 6)	2.0
  (6, 1)	3.0
  (6, 5)	1.0
  (6, 7)	2.0
  (7, 2)	3.0
  (7, 6)	1.0
  (7, 8)	2.0
  (8, 3)	3.0
  (8, 7)	1.0
  (8, 9)	2.0
  (9, 4)	3.0
  (9, 8)	1.0


In [9]:
import json
with open("../../IM-TQA/data/train_tables copy.json") as f:
    data1 = json.load(f)
with open("../../IM-TQA/data/dev_tables.json") as f:
    data2 = json.load(f)
with open("../../IM-TQA/data/test_tables.json") as f:
    data3 = json.load(f)
data = data1 + data2 + data3
with open("../../IM-TQA/data/train_dev_test.json", "w") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [4]:
from __future__ import print_function

from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from rgcn.layers.graph import GraphConvolution
from rgcn.layers.input_adj import InputAdj
from rgcn.utils import *

import pickle as pkl

import os
import sys
import time
import argparse
import numpy as np

split = 29205


def get_split(y, train_idx, test_idx, validation=True):
    # Make dataset splits
    # np.random.shuffle(train_idx)
    if validation:
        idx_train = train_idx[:split]
        idx_val = train_idx[split:]
        idx_test = idx_val  # report final score on validation set for hyperparameter optimization
    else:
        idx_train = train_idx
        idx_val = train_idx  # no validation
        idx_test = test_idx

    y_train = np.zeros(y.shape)
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)

    y_train[idx_train] = np.array(y[idx_train].todense())
    y_val[idx_val] = np.array(y[idx_val].todense())
    y_test[idx_test] = np.array(y[idx_test].todense())

    return y_train, y_val, y_test, idx_train, idx_val, idx_test


np.random.seed()

# ap = argparse.ArgumentParser()
# ap.add_argument(
#     "-d",
#     "--dataset",
#     type=str,
#     default="aifb",
#     help="Dataset string ('aifb', 'mutag', 'bgs', 'am')",
# )
# ap.add_argument("-e", "--epochs", type=int, default=50, help="Number training epochs")
# ap.add_argument("-hd", "--hidden", type=int, default=16, help="Number hidden units")
# ap.add_argument("-do", "--dropout", type=float, default=0.0, help="Dropout rate")
# ap.add_argument(
#     "-b", "--bases", type=int, default=-1, help="Number of bases used (-1: all)"
# )
# ap.add_argument("-lr", "--learnrate", type=float, default=0.01, help="Learning rate")
# ap.add_argument(
#     "-l2", "--l2norm", type=float, default=0.0, help="L2 normalization of input weights"
# )

# fp = ap.add_mutually_exclusive_group(required=False)
# fp.add_argument("--validation", dest="validation", action="store_true")
# fp.add_argument("--testing", dest="validation", action="store_false")
# ap.set_defaults(validation=True)
# args = vars(ap.parse_args())
# print(args)

# Define parameters
DATASET = "table_val"
NB_EPOCH = 20
VALIDATION = True
LR = 0.01
L2 = 0.0
HIDDEN = 800
BASES = 0
DO = 0.0

with open(DATASET + ".pickle", "rb") as f:
    data = pkl.load(f)

A = data["A"]
y = data["y"]
train_idx = data["train_idx"]
test_idx = data["test_idx"]

# Get dataset splits
y_train, y_val, y_test, idx_train, idx_val, idx_test = get_split(
    y, train_idx, test_idx, VALIDATION
)
train_mask = sample_mask(idx_train, y.shape[0])

num_nodes = A[0].shape[0]
support = len(A)

# Define empty dummy feature matrix (input is ignored as we set featureless=True)
# In case features are available, define them here and set featureless=False.
X = sp.csr_matrix(A[0].shape)

# Normalize adjacency matrices individually
for i in range(len(A)):
    d = np.array(A[i].sum(1)).flatten()
    d_inv = 1.0 / d
    d_inv[np.isinf(d_inv)] = 0.0
    D_inv = sp.diags(d_inv)
    A[i] = D_inv.dot(A[i]).tocsr()

A_in = [InputAdj(sparse=True) for _ in range(support)]
X_in = Input(shape=(X.shape[1],), sparse=True)

# Define model architecture
H = GraphConvolution(
    HIDDEN,
    support,
    num_bases=BASES,
    featureless=True,
    activation="relu",
    W_regularizer=l2(L2),
)([X_in] + A_in)
H = Dropout(DO)(H)
Y = GraphConvolution(y_train.shape[1], support, num_bases=BASES, activation="softmax")(
    [H] + A_in
)

# Compile model
model = Model(input=[X_in] + A_in, output=Y)
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=LR))

preds = None

# Fit
for epoch in range(1, NB_EPOCH + 1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration
    model.fit(
        [X] + A,
        y_train,
        sample_weight=train_mask,
        batch_size=num_nodes,
        nb_epoch=1,
        shuffle=False,
        verbose=0,
    )

    if epoch % 1 == 0:

        # Predict on full dataset
        preds = model.predict([X] + A, batch_size=num_nodes)

        # Train / validation scores
        train_val_loss, train_val_acc = evaluate_preds(
            preds, [y_train, y_val], [idx_train, idx_val]
        )

        print(
            "Epoch: {:04d}".format(epoch),
            "train_loss= {:.4f}".format(train_val_loss[0]),
            "train_acc= {:.4f}".format(train_val_acc[0]),
            "val_loss= {:.4f}".format(train_val_loss[1]),
            "val_acc= {:.4f}".format(train_val_acc[1]),
            "time= {:.4f}".format(time.time() - t),
        )

    else:
        print("Epoch: {:04d}".format(epoch), "time= {:.4f}".format(time.time() - t))

# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(
    "Test set results:",
    "loss= {:.4f}".format(test_loss[0]),
    "accuracy= {:.4f}".format(test_acc[0]),
)

Epoch: 0001 train_loss= 0.5878 train_acc= 0.6633 val_loss= 1.3266 val_acc= 0.4771 time= 58.8309
Epoch: 0002 train_loss= 0.2405 train_acc= 0.6646 val_loss= 1.2963 val_acc= 0.4897 time= 54.4621
Epoch: 0003 train_loss= 0.0883 train_acc= 0.6745 val_loss= 1.2761 val_acc= 0.5206 time= 56.8243
Epoch: 0004 train_loss= 0.0293 train_acc= 0.7003 val_loss= 1.2612 val_acc= 0.5480 time= 55.5610
Epoch: 0005 train_loss= 0.0103 train_acc= 0.7311 val_loss= 1.2496 val_acc= 0.5892 time= 55.5088
Epoch: 0006 train_loss= 0.0048 train_acc= 0.7592 val_loss= 1.2401 val_acc= 0.6284 time= 56.7165
Epoch: 0007 train_loss= 0.0032 train_acc= 0.7805 val_loss= 1.2321 val_acc= 0.6551 time= 57.3918
Epoch: 0008 train_loss= 0.0027 train_acc= 0.7978 val_loss= 1.2253 val_acc= 0.6774 time= 55.2069
Epoch: 0009 train_loss= 0.0025 train_acc= 0.8121 val_loss= 1.2196 val_acc= 0.6919 time= 54.9180
Epoch: 0010 train_loss= 0.0025 train_acc= 0.8236 val_loss= 1.2147 val_acc= 0.7030 time= 55.4031
Epoch: 0011 train_loss= 0.0024 train_acc

In [1]:
model.save("my_model.h5")

NameError: name 'model' is not defined

In [4]:
import json
with open("E:\FDU\Grade_4\Term_2\毕业论文\IM-TQA\data\dev_tables.json","r",encoding="utf-8") as f:
    json_data=json.load(f)
for each in json_data:
    if each["table_id"] == "OPgtm8om":
        print(each)

{'table_id': 'OPgtm8om', 'table_type': 'vertical', 'file_name': '垂直表格_34', 'cell_ID_matrix': [[0, 1, 2], [3, 4, 5], [6, 7, 8]], 'chinese_cell_value_list': ['项   目', '2021年度', '2020年度', '坏账损失', '-1,467,790.73', '-700,450.60', '合计', '-1,467,790.73', '-700,450.60'], 'english_cell_value_list': ['project', 'Year 2021', 'Year 2020', 'Bad debt loss', '-1,467,790.73', '-700,450.60', 'total', '-1,467,790.73', '-700,450.60'], 'column_attribute': [0, 1, 2], 'row_attribute': [6], 'column_index': [], 'row_index': [3]}


In [2]:
import requests
import json

# API 端点
url = "http://10.176.40.88:8010/embed"

# 请求数据
data = {
    "texts": [
        "This is the first text.",
        "Here is another text.",
        "This is yet another example text."
    ]
}

# 设置请求头
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# 发送 POST 请求
response = requests.post(url, headers=headers, data=json.dumps(data))

# 检查响应状态
if response.status_code == 200:
    # 打印嵌入结果
    embeddings = response.json()
    print(json.dumps(embeddings, indent=4))
else:
    # 打印错误信息
    print(f"Error: {response.status_code}")
    print(response.text)

ConnectionError: HTTPConnectionPool(host='10.176.40.88', port=8010): Max retries exceeded with url: /embed (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2ecadccd50>: Failed to establish a new connection: [Errno 111] Connection refused'))